# transformation을 할건데 select만 사용할 상황
- 원본데이터를 캐시에 넣어두고 활용하자!
-> 일반적인 활용법

In [1]:
import findspark
findspark.init()

In [2]:
from pyspark import SparkConf, SparkContext
conf = SparkConf().setMaster("local").setAppName("cache_persist_example")
sc = SparkContext(conf=conf)

In [4]:
directory='c://Users//apfhd//SparkExam//data//restaurant_reviews.csv'

In [5]:
lines = sc.textFile(f"file:///{directory}")
lines.collect()

['id,item,cateogry,reviews,',
 '0,짜장면,중식,125,',
 '1,짬뽕,중식,235,',
 '2,김밥,분식,32,',
 '3,떡볶이,분식,534,',
 '4,라멘,일식,223,',
 '5,돈가스,일식,52,',
 '6,우동,일식,12,',
 '7,쌀국수,아시안,312,',
 '8,햄버거,패스트푸드,12,',
 '9,치킨,패스트푸드,23']

### 조회 대상이 될 만한 데이터만 cache,persist에 넣는다! 
==> 컬럼도 물론 제거!

In [7]:
header=lines.first()
rows=lines.filter(lambda row : row !=header)
rows.collect()

['0,짜장면,중식,125,',
 '1,짬뽕,중식,235,',
 '2,김밥,분식,32,',
 '3,떡볶이,분식,534,',
 '4,라멘,일식,223,',
 '5,돈가스,일식,52,',
 '6,우동,일식,12,',
 '7,쌀국수,아시안,312,',
 '8,햄버거,패스트푸드,12,',
 '9,치킨,패스트푸드,23']

In [8]:
def parse(row):
    fields=row.split(',')
    category=fields[2]
    reviews=int(fields[3])
    return category,reviews

## persist를 사용하지 않는 경우 ( 대부분 persist를 사용한다)

In [17]:
# 비즈니스에 연관된 transformations를 수행할 RDD 생성
# 카테고리와 리뷰수에 대한 것만 활용할 때
categoryReviews = rows.map(parse)
categoryReviews.collect()

[('중식', 125),
 ('중식', 235),
 ('분식', 32),
 ('분식', 534),
 ('일식', 223),
 ('일식', 52),
 ('일식', 12),
 ('아시안', 312),
 ('패스트푸드', 12),
 ('패스트푸드', 23)]

In [18]:
result1 = categoryReviews.take(5)
result1

[('중식', 125), ('중식', 235), ('분식', 32), ('분식', 534), ('일식', 223)]

In [19]:
result2 = categoryReviews.mapValues(lambda x : (x, 1)).collect()
result2

[('중식', (125, 1)),
 ('중식', (235, 1)),
 ('분식', (32, 1)),
 ('분식', (534, 1)),
 ('일식', (223, 1)),
 ('일식', (52, 1)),
 ('일식', (12, 1)),
 ('아시안', (312, 1)),
 ('패스트푸드', (12, 1)),
 ('패스트푸드', (23, 1))]

`categoryReviews`는 `result1`, `result2`를 수행하면서 두 개가 만들어 진다.  
` 매번 활용 할 때마다 RDD가 생성되어 메모리를 잡아먹는다.`

`categoryReviews`에서 데이터를 꺼내오기만 하면 되지, 변경은 일어나지 않기 때문에  
 `persist`를 이용해서 `categoryReviews`를 메모리에 넣어 놓는 것이 조금 더 유리할 것이다.

## `persist`를 사용하는 경우

In [20]:
# categoryReviews만 사용할 것이며 변형이 없을 것이기에 persist로 하나만 RDD를 만든다.
categoryReviews_persist = rows.map(parse).persist() # categoryReviews RDD는 메모리 상에 하나만 존재하는 RDD가 된다.
categoryReviews_persist

PythonRDD[16] at RDD at PythonRDD.scala:53

In [21]:
result3 = categoryReviews_persist.take(5)
result3

[('중식', 125), ('중식', 235), ('분식', 32), ('분식', 534), ('일식', 223)]

In [22]:
result4 = categoryReviews_persist.mapValues(lambda x : (x, 1)).collect()
result4

[('중식', (125, 1)),
 ('중식', (235, 1)),
 ('분식', (32, 1)),
 ('분식', (534, 1)),
 ('일식', (223, 1)),
 ('일식', (52, 1)),
 ('일식', (12, 1)),
 ('아시안', (312, 1)),
 ('패스트푸드', (12, 1)),
 ('패스트푸드', (23, 1))]